Featch API key

In [1]:
import os
from dotenv import load_dotenv
from groq import Groq

In [2]:
load_dotenv()  
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not found in .env file!")

client = Groq(api_key=GROQ_API_KEY)

Generate a cover letter using llama-3.1-8b-instant

In [26]:
import json
import re

## Huge Prompt 

In [55]:
def generate_cover_letter_parts(user_job_json: dict) -> dict:

    user = user_job_json["user"]
    job = user_job_json["job"]
    #print("user ", user)
    #print("job ", job)

    prompt = f"""
You are an expert cover letter writer. Based on the following structured user and job information in JSON format, generate three professional and engaging paragraphs of a cover letter: Opening, Body, and Closing.

1. **Opening Paragraph**:
   In the opening paragraph tell how you learned about the position. You may, for example, know of a job through: Qaddemly website


2. **Body Paragraph**:

    This paragraph gives a summary of your background and critical skills (hard skills) that make you qualified for the position.
    This paragraph can be used to demonstrate your persuasive skills (soft skills).

3. **Closing Paragraph**:
   At the end of the letter talk about your availability for the job, where you can be contacted, and when you are going to contact the hiring person for an appointment to discuss your application. If you have no contact name you may simply want to indicate your anticipation for a response in this part of the letter. Thank the person to whom you are writing for his/her time and consideration of your application.

**User information:**
{user}

**Job information:**
{job}
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.6,
        max_tokens=600,
    )

    content = response.choices[0].message.content.strip()
    content = re.sub(r"\*\*.*?Paragraph:\*\*", "", content, flags=re.IGNORECASE).strip()
    paragraphs = [p.strip() for p in content.split("\n\n") if p.strip()]

    # Concatenate the paragraphs with newlines between them
    merged_content = "\n\n".join(paragraphs)

    return {"merged_cover_letter": merged_content}


In [57]:
# Example Usage:
import json

with open("data/test.json", "r") as f:
    user_job_json = json.load(f)

result = generate_cover_letter_parts(user_job_json)
print(result['merged_cover_letter'])

I am writing to express my enthusiastic interest in the Digital Marketing Specialist position at Vodafone Haha, as advertised on Qaddemly. With a strong background in computer science and experience in various roles, I am confident that my unique blend of skills and passion for innovation make me an ideal candidate for this exciting opportunity. I am particularly drawn to Vodafone Haha's commitment to excellence and its reputation for pushing the boundaries of digital marketing.

As a highly motivated and experienced professional, I possess a strong foundation in computer science, with a degree in Computer Science from Cairo University. My skills in JavaScript, React, and Node.js have enabled me to excel in various roles, including Frontend and Backend Development positions at Tech Solutions Inc. and Innovate Tech, respectively. I am confident that my technical expertise, combined with my passion for digital marketing, make me an excellent fit for this position. I am excited about the 

['Here are three professional and engaging paragraphs for a cover letter:', '**Opening Paragraph**', 'I am writing to express my interest in the Digital Marketing Specialist position at Vodafone Haha, which I came across on Qaddemly website. As a highly motivated and experienced professional in the field of digital marketing, I am confident that I would be a valuable asset to your team. I am excited about the opportunity to contribute my skills and expertise to help drive brand awareness and engagement for your organization.', '**Body Paragraph**', 'With over 3 years of experience in the tech industry, including roles as a Frontend Developer at Tech Solutions Inc. and a Backend Developer at Innovate Tech, I possess a unique blend of technical and creative skills that would be highly beneficial in this position. My proficiency in JavaScript, React, and Node.js has equipped me with a solid understanding of web development principles, which I believe would be an asset in creating engaging content and analyzing social media metrics. Moreover, my experience working with various social media platforms has given me a deep understanding of community engagement and the importance of creating content that resonates with diverse audiences.', '**Closing Paragraph**', 'I am excited about the opportunity to discuss my application and how I can contribute to the success of Vodafone Haha as a Digital Marketing Specialist. I am available for an interview at your earliest convenience and can be reached at abdotest@mailsac.com or via phone at [insert phone number]. I would like to thank you for considering my application and look forward to the opportunity to discuss this position further. I am confident that my skills, experience, and passion for digital marketing make me an ideal candidate for this role, and I am eager to learn more about the position and how I can contribute to the team.']

## after merging the paragraphs

I am writing to express my enthusiastic interest in the Digital Marketing Specialist position at Vodafone Haha, as advertised on Qaddemly. With a strong background in computer science and experience in various roles, I am confident that my unique blend of skills and passion for innovation make me an ideal candidate for this exciting opportunity. I am particularly drawn to Vodafone Haha's commitment to excellence and its reputation for pushing the boundaries of digital marketing.

As a highly motivated and experienced professional, I possess a strong foundation in computer science, with a degree in Computer Science from Cairo University. My skills in JavaScript, React, and Node.js have enabled me to excel in various roles, including Frontend and Backend Development positions at Tech Solutions Inc. and Innovate Tech, respectively. I am confident that my technical expertise, combined with my passion for digital marketing, make me an excellent fit for this position. I am excited about the prospect of applying my skills to drive brand awareness and engagement through social media platforms, content creation, and analytics.

Thank you for considering my application for the Digital Marketing Specialist position at Vodafone Haha. I would be delighted to discuss my qualifications further and learn more about your team's work. Please feel free to contact me at abdotest@mailsac.com or through my LinkedIn profile. I look forward to the opportunity to meet with you and explore how my skills and experience can contribute to the success of Vodafone Haha. I am available to start immediately and am eager to learn more about your team's work. Thank you again for your time and consideration.

Note: I assumed that the CEO of Vodafone Haha is Abdo Khattab based on the job information provided. If that's not the case, please let me know and I'll adjust the letter accordingly.

## simple Prompt 

In [33]:
def generate_cover_letter_parts(user_job_json: dict) -> dict:

    user = user_job_json["user"]
    job = user_job_json["job"]
    business = job.get("business", {})

    prompt = f"""
You are an expert cover letter writer. Based on the following structured user and job information in JSON format, generate three professional and engaging paragraphs of a cover letter: Opening, Body, and Closing.

Only return the paragraphs. Do not label them with headings like 'Opening Paragraph', 'Body Paragraph', or 'Closing Paragraph'. Just provide each paragraph with a line break in between.

**User infomation:**
{user}

**Job information:**
{job}
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.6,
        max_tokens=600,
    )

    content = response.choices[0].message.content.strip()

    # Remove headers if they exist, just in case model still includes them
    content = re.sub(r"\*\*.*?Paragraph:\*\*", "", content, flags=re.IGNORECASE).strip()

    # Split paragraphs
    paragraphs = [p.strip() for p in content.split("\n\n") if p.strip()]

    return {
        "opening": paragraphs[0] if len(paragraphs) > 0 else "",
        "body": paragraphs[1] if len(paragraphs) > 1 else "",
        "closing": paragraphs[2] if len(paragraphs) > 2 else "",
    }



In [34]:
# Example Usage:
import json

with open("data/test.json", "r") as f:
    user_job_json = json.load(f)

result = generate_cover_letter_parts(user_job_json)

print("\n", result["opening"])
print("\n\n", result["body"])
print("\n\n", result["closing"])


 Dear Hiring Manager,


 I am excited to apply for the Digital Marketing Specialist position at Vodafone Haha, as advertised. With my background in Computer Science and experience in various technical roles, I am confident that I can bring a unique perspective to this position and contribute to the company's success. Specifically, my skills in JavaScript, React, and Node.js have allowed me to develop a strong understanding of the digital landscape, and I am eager to apply this knowledge in a marketing context.


 As a skilled developer, I have a proven track record of creating engaging content and interacting with online communities. In my current role as Backend Developer at Innovate Tech, I have developed a strong understanding of social media analytics and insights, which I believe would be highly valuable in this position. Additionally, my experience working with various social media platforms has given me a deep understanding of their capabilities and limitations, and I am confid

Dear Hiring Manager,


 I am excited to apply for the Digital Marketing Specialist position at Vodafone Haha, as advertised. With my background in Computer Science and experience in various technical roles, I am confident that I can bring a unique perspective to this position and contribute to the company's success. Specifically, my skills in JavaScript, React, and Node.js have allowed me to develop a strong understanding of the digital landscape, and I am eager to apply this knowledge in a marketing context.


 As a skilled developer, I have a proven track record of creating engaging content and interacting with online communities. In my current role as Backend Developer at Innovate Tech, I have developed a strong understanding of social media analytics and insights, which I believe would be highly valuable in this position. Additionally, my experience working with various social media platforms has given me a deep understanding of their capabilities and limitations, and I am confident that I can leverage this knowledge to drive brand awareness and engagement for Vodafone Haha.